In [93]:
import pandas as pd
import matplotlib.pyplot as plt
df = pandas.read_csv("bitcoin_v2.csv")
df = df.drop(['Unnamed: 0'], axis = 1)
df

,date,confirmation_time,difficulty,hash_rate,market_cap,price_usd,total_block_size,total_miner_revenue,total_supply,total_trx_fees,trade_volume,transactions,trx_cost_percent,trx_per_block,trx_volume,unique_addresses,daily_return
0,2015-01-01,7.150000,4.064096e+10,3.353653e+05,4.317111e+09,315.70000,27835.0000,1312680.60,13674725.0,8.003076,5.553875e+06,58963,5.244721,555.000000,79280.0000,116012,0
1,2015-01-02,6.933333,4.064096e+10,3.232437e+05,4.324529e+09,316.15000,27875.0000,1267761.50,13678725.0,10.727695,6.440896e+06,72234,4.477312,627.000000,89563.0000,168563,0
2,2015-01-03,6.433333,4.064096e+10,3.313247e+05,4.136728e+09,302.33000,27922.0000,1242878.63,13682825.0,11.906088,2.938312e+07,82832,2.277049,734.000000,180541.0000,207571,0
3,2015-01-04,8.033333,4.064096e+10,3.353653e+05,3.708212e+09,270.93000,27971.0000,1129778.10,13686975.0,20.288107,5.310479e+07,85570,1.376066,692.000000,303038.0000,206228,0
4,2015-01-05,6.816667,4.064096e+10,3.394058e+05,3.789717e+09,276.80000,28024.0000,1166988.80,13691175.0,16.967084,3.132672e+07,93753,1.863192,751.000000,226278.0000,193982,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,2017-12-27,8.133333,1.873110e+12,1.452556e+07,2.613810e+11,15589.32167,148457.8829,41529952.92,16766700.0,784.817123,1.619269e+09,307642,0.908155,1972.064103,301139.8528,749191,0
1089,2017-12-28,9.800000,1.873110e+12,1.387377e+07,2.411420e+11,14380.58167,148619.2801,37008426.92,16768562.5,785.833892,1.883494e+09,305053,1.052555,2047.335570,251609.9899,741488,0
1090,2017-12-29,8.066667,1.873110e+12,1.536358e+07,2.455240e+11,14640.14000,148792.1595,40531227.59,16770625.0,785.795050,1.274049e+09,353824,1.132235,2144.387879,251563.8573,838925,0
1091,2017-12-30,10.700000,1.873110e+12,1.368755e+07,2.216580e+11,13215.57400,148944.5812,33640243.62,16772462.5,778.571110,1.728482e+09,344407,0.876878,2342.904762,298339.3935,815099,0


# Slicing our data

In [94]:
df['date'] = pd.to_datetime(df['date'])
df.set_axis(df['date'], inplace=True)
df.drop(columns=['confirmation_time','difficulty','hash_rate','market_cap','total_block_size',
                         'total_miner_revenue','total_supply','total_trx_fees','trade_volume',
                         'transactions','trx_cost_percent','trx_per_block','trx_volume',
                         'unique_addresses','daily_return'], inplace=True)
df

,date,price_usd
date,,
2015-01-01,2015-01-01,315.70000
2015-01-02,2015-01-02,316.15000
2015-01-03,2015-01-03,302.33000
2015-01-04,2015-01-04,270.93000
2015-01-05,2015-01-05,276.80000
...,...,...
2017-12-27,2017-12-27,15589.32167
2017-12-28,2017-12-28,14380.58167
2017-12-29,2017-12-29,14640.14000


In [95]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Data PreProcessing

In [96]:
price_data = df['price_usd'].values
price_data  = price_data .reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(price_data ))

price_train = price_data [:split]
price_test = price_data [split:]

date_train = df['date'][:split]
date_test = df['date'][split:]

print(len(price_train))
print(len(price_test))

874
219


# Using previous 30 days to train

In [102]:
from keras.preprocessing.sequence import TimeseriesGenerator
look_back = 30

train_generator = TimeseriesGenerator(price_train, price_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(price_test, price_test, length=look_back, batch_size=1)


# Building the model

In [103]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()

model.add(LSTM(10,activation='relu',input_shape=(look_back,1)))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25


<ipython-input-103-3c28133074e3>:13: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



43/43 [==============================] - 2s 16ms/step - loss: 539594.3750
Epoch 2/25
43/43 [==============================] - 1s 16ms/step - loss: 466775.5000
Epoch 3/25
43/43 [==============================] - 1s 17ms/step - loss: 455896.5938
Epoch 4/25
43/43 [==============================] - 1s 16ms/step - loss: 437796.8125
Epoch 5/25
43/43 [==============================] - 1s 16ms/step - loss: 468599.4688
Epoch 6/25
43/43 [==============================] - 1s 16ms/step - loss: 453461.5000
Epoch 7/25
43/43 [==============================] - 1s 16ms/step - loss: 454501.4688
Epoch 8/25
43/43 [==============================] - 1s 16ms/step - loss: 452028.1562
Epoch 9/25
43/43 [==============================] - 1s 16ms/step - loss: 453076.7812
Epoch 10/25
43/43 [==============================] - 1s 16ms/step - loss: 452852.0312
Epoch 11/25
43/43 [==============================] - 1s 19ms/step - loss: 451642.4688
Epoch 12/25
43/43 [==============================] - 1s 15ms/step - loss: 

# Prediction

In [104]:
import plotly.graph_objs as go

In [105]:

prediction = model.predict_generator(test_generator)

price_train = price_train.reshape((-1))
price_test = price_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = price_train,
    mode = 'lines',
    name = 'Data'
)
trace3 = go.Scatter(
    x = date_test,
    y = price_test,
    mode='lines',
    name = 'Ground Truth'
)

trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "Bitcoin price",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Price_USD"}
)
fig = go.Figure(data=[trace1, trace3, trace2], layout=layout)
fig.show()

<ipython-input-105-6f6500dd1180>:1: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



# Future Forecasting

In [31]:
import numpy as np

price_data = price_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = price_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 90
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

# Visualize Future Prediction

In [32]:
#prediction = model.predict_generator(test_generator)

price_train = price_train.reshape((-1))
price_test = price_test.reshape((-1))
forecast = forecast.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = price_train,
    mode = 'lines',
    name = 'Data'
)

trace2 = go.Scatter(
    x = date_test,
    y = price_test,
    mode='lines',
    name = 'Ground Truth'
)

trace3 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "Bitcoin price",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Price_USD"}
)
fig = go.Figure(data=[trace1,trace2,trace3], layout=layout)
fig.show()